In [1]:
from docplex.mp.model import Model
from docplex.mp.relax_linear import LinearRelaxer

def make_bus_model():
    mdl = Model(name='buses')
    A = [(i,j) for i in range(4) for j in range(4) if i!=j]
    print(A)
    nbbus40 = mdl.binary_var_dict(A,name='nbBus40')
    nbbus30 = mdl.integer_var(name='nbBus30')

    mdl.add_constraint(mdl.sum(nbbus40[i,j] * 40 for i,j in A) + nbbus30 * 30 >= 300, 'kids')
    mdl.minimize(mdl.sum(nbbus40[i,j] * 500 + nbbus30 * 400 for i,j in A))
    return mdl

if __name__ == '__main__':
    bm1 = make_bus_model()
    bm1.print_information()
    s1 = bm1.solve(log_output=True)
    s1.display()

    bmr = LinearRelaxer.make_relaxed_model(bm1, return_partial=True)
    bmr.print_information()
    rs = bmr.solve(log_output=True)
    rs.display()

    duals = bmr.get_constraint_by_name("kids").dual_value

    print("dual of the 300 kids constraint = ",duals) 

[(0, 1), (0, 2), (0, 3), (1, 0), (1, 2), (1, 3), (2, 0), (2, 1), (2, 3), (3, 0), (3, 1), (3, 2)]
Model: buses
 - number of variables: 13
   - binary=12, integer=1, continuous=0
 - number of constraints: 1
   - linear=1
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP
Version identifier: 20.1.0.0 | 2020-11-10 | 9bedb6d68
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 48000.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 11 columns.
MIP Presolve added 1 rows and 1 columns.
MIP Presolve modified 1 coefficients.
Reduced MIP has 1 rows, 3 columns, and 3 nonzeros.
Reduced MIP has 0 binaries, 3 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.01 ticks)
Tried aggregator 1 time.
MIP Presolve eliminated 1 rows and 1 columns.
MIP Presolve added 1 rows and 1 columns.
Reduced MIP has 1 rows, 3 columns, and 3 nonzeros.
Reduced MIP has 0 binaries, 3 generals, 0 SOSs, and 0 indicators.
P

In [2]:
def alterVariablesFor(mdl, variables, type_wish):
    relaxed_model = mdl.copy()
    model_name = mdl.name
    mdl_class = mdl.__class__
    relax_model_name = 'rlx_'+mdl.name
    relaxed_model = mdl_class(name=relax_model_name)
    # transfer variable containers
    ctn_map = {}
    for ctn in mdl.iter_var_containers():
        copied_ctn = ctn.copy_relaxed(relaxed_model)
        ctn_map[ctn] = copied_ctn
    # transfer variables    
    
    var_mapping = {}
    for v in mdl.iter_variables():
        if v.name in variables:
            rx_lb = v.lb
            copied_var = relaxed_model._var(type_wish, rx_lb, v.ub, v.name)
            var_ctn = v.container
            if var_ctn:
                copied_ctn = ctn_map.get(var_ctn)
                assert copied_ctn is not None
                copied_var.container = copied_ctn
            var_mapping[v] = copied_var
        else:
            copied_var = relaxed_model._var(v._vartype, v.lb, v.ub, v.name)
            var_ctn = v.container
            if var_ctn:
                copied_ctn = ctn_map.get(var_ctn)
                assert copied_ctn is not None
                copied_var.container = copied_ctn
            var_mapping[v] = copied_var
            
    # transfer all non-logical cts
    for ct in mdl.iter_constraints():
        if not ct.is_generated():
            if ct.is_logical():
                process_unrelaxable(ct, 'logical')
            try:
                copied_ct = ct.relaxed_copy(relaxed_model, var_mapping)
                relaxed_model.add(copied_ct)
            except DocplexLinearRelaxationError as xe:
                process_unrelaxable(xe.object, xe.cause)
            except KeyError as ke:
                info('failed to relax constraint: {0}'.format(ct))
                process_unrelaxable(ct, 'key')
    # clone objective
    relaxed_model.objective_sense = mdl.objective_sense
    try:
        relaxed_model.objective_expr = mdl.objective_expr.relaxed_copy(relaxed_model, var_mapping)
    except DocplexLinearRelaxationError as xe:
        process_unrelaxable(urx_=xe.object, reason=xe.cause)
    except KeyError:
        process_unrelaxable(urx_=mdl.objective_expr, reason='objective')
    if mdl.context:
        relaxed_model.context = mdl.context.copy()
    return relaxed_model

In [5]:
A = [(i,j) for i in range(4) for j in range(4) if i!=j]

In [4]:
bm1.print_information()

Model: buses
 - number of variables: 13
   - binary=12, integer=1, continuous=0
 - number of constraints: 1
   - linear=1
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [4]:
variables = ['nbBus40_0_1','nbBus40_0_2','nbBus40_0_3']
bmr2 = alterVariablesFor(bm1, variables, bm1.continuous_vartype)

In [5]:
bmr2.print_information()

Model: rlx_buses
 - number of variables: 13
   - binary=9, integer=1, continuous=3
 - number of constraints: 1
   - linear=1
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [8]:
bmi3 = alterVariablesFor(bmr2, variables, bm1.binary_vartype)

In [9]:
bmi3.print_information()

Model: rlx_rlx_buses
 - number of variables: 13
   - binary=12, integer=1, continuous=0
 - number of constraints: 1
   - linear=1
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [13]:
bm1.binary_vartype